In [1]:
import numpy as np
import pandas as pd
import os 
import time
import deepdish.io as dd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

from svi import SVI
import models as m
import datasets as d

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
data = dd.load("sequenced_data_for_VAE_lenght-160_stride-10.h5")

In [4]:
X,y = data["X"], data["y"]

In [5]:
X.shape

torch.Size([83680, 160])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666, shuffle=True)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
X_train = torch.tensor(X_train.astype('float32'))
X_test = torch.tensor(X_test.astype('float32'))

# standartní VAE

In [10]:
VAE = torch.load("VAE-saved_model_and_losses/DeepDenseVAE_mark_V_[160-256-128-15]_VDO_GNLL_400ep_lr-1e-4.pt")

In [11]:
VAE.eval()

DeepDenseVAE_mark_V(
  (encoder): Sequential(
    (0): Linear(in_features=160, out_features=256, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ELU(alpha=1.0)
    (4): VariationalLayer(
      (mu): Linear(in_features=128, out_features=15, bias=True)
      (rho): Linear(in_features=128, out_features=15, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=15, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ELU(alpha=1.0)
    (4): VariationalDecoderOutput(
      (mu): Linear(in_features=256, out_features=160, bias=True)
      (rho): Linear(in_features=256, out_features=1, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
)

In [12]:
Z_hat, mu_tr, sigma_tr = VAE.encoder(X_train.to(device))
mu_train = mu_tr.cpu().detach().numpy()

Z_hat, mu_te, sigma_te = VAE.encoder(X_test.to(device))
mu_test = mu_te.cpu().detach().numpy()

In [14]:
VAEclf = GradientBoostingClassifier(n_estimators=400, learning_rate=0.1, subsample=0.7, verbose=True)

In [15]:
VAEclf.fit(mu_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       40066.7726        2203.7504            6.55m
         2       37449.3970        1109.7092            6.43m
         3       35414.2218         847.0231            6.33m
         4       33670.6681         667.6218            6.26m
         5       32467.0183         558.1413            6.25m
         6       31426.2867         436.8396            6.28m
         7       30469.5283         400.8426            6.25m
         8       29693.1123         355.1634            6.26m
         9       29012.5904         280.1077            6.25m
        10       28551.9408         237.5730            6.21m
        20       24798.5412          83.0547            6.05m
        30       22811.8798          70.3154            5.92m
        40       21530.0274          46.0703            5.78m
        50       20906.6638          19.1692            5.61m
        60       20306.7768          17.4899            5.48m
       

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=400,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [17]:
y_pred = VAEclf.predict(mu_test)

In [18]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2887, 1188,   23,  137],
       [ 404, 9881,   36,  124],
       [  25,   47,  391,   16],
       [ 171,  192,   17, 1197]], dtype=int64)

In [19]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.857791586998088

In [20]:
dd.save("VAE-saved_model_and_losses/GB_on_VAE_latent_space.h5", VAEclf)

# CNN-VAE (elu)

In [21]:
CNNVAE1 = torch.load("VAE-saved_model_and_losses/DeepConvVAE_ELU_NLL_250ep_lr-{1e-3, 1e-4}_bs-300.pt")

In [22]:
CNNVAE1.eval()

DeepConvVAE_ELU(
  (encoder): Sequential(
    (0): Reshape()
    (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,))
    (2): ELU(alpha=1.0)
    (3): Conv1d(16, 32, kernel_size=(4,), stride=(2,))
    (4): ELU(alpha=1.0)
    (5): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
    (6): ELU(alpha=1.0)
    (7): Flatten()
    (8): VariationalLayer(
      (mu): Linear(in_features=1152, out_features=15, bias=True)
      (rho): Linear(in_features=1152, out_features=15, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=15, out_features=1152, bias=True)
    (1): ELU(alpha=1.0)
    (2): Reshape()
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,))
    (4): ELU(alpha=1.0)
    (5): ConvTranspose1d(32, 16, kernel_size=(4,), stride=(2,))
    (6): ELU(alpha=1.0)
    (7): ConvTransposeDecoderOutput(
      (mu): ConvTranspose1d(16, 1, kernel_size=(6,), stride=(2,))
      (rho): Linear(in_features=1248, out_features=1, bias

In [23]:
Z_hat, mu_tr, sigma_tr = CNNVAE1.encoder(X_train.to(device))
mu_train = mu_tr.cpu().detach().numpy()

Z_hat, mu_te, sigma_te = CNNVAE1.encoder(X_test.to(device))
mu_test = mu_te.cpu().detach().numpy()

In [24]:
CNNVAE1clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, subsample=0.7, verbose=True)

In [25]:
CNNVAE1clf.fit(mu_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       40008.1169        2207.1310            8.10m
         2       37309.4551        1151.5303            8.32m
         3       35309.5284         890.4406            8.30m
         4       33712.1222         673.6487            8.36m
         5       32215.0148         546.1065            8.32m
         6       31326.7429         459.3907            8.33m
         7       30394.1315         367.0631            8.27m
         8       29688.5431         335.8347            8.26m
         9       28608.0708         328.4486            8.23m
        10       28180.3092         275.7602            8.20m
        20       24241.4459          97.3563            8.01m
        30       22519.4545          44.2201            7.83m
        40       21643.5372          25.3989            7.60m
        50       20761.7821          27.9019            7.43m
        60       20169.1706          17.2932            7.27m
       

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [26]:
y_pred = CNNVAE1clf.predict(mu_test)

In [27]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2806, 1262,   30,  137],
       [ 419, 9874,   45,  107],
       [  29,   42,  393,   15],
       [ 181,  170,   17, 1209]], dtype=int64)

In [28]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8533699808795411

In [29]:
dd.save("VAE-saved_model_and_losses/GB_on_CNNVAE-elu_latent_space.h5", CNNVAE1clf)

# CNN VAE (relu)

In [9]:
CNNVAE2 = torch.load("VAE-saved_model_and_losses/DeepConvVAE_ReLU_NLL_200ep_lr-1e-4_bs-100.pt")

In [10]:
CNNVAE2.eval()

DeepConvVAE(
  (encoder): Sequential(
    (0): Reshape()
    (1): Conv1d(1, 16, kernel_size=(6,), stride=(2,))
    (2): ReLU()
    (3): Conv1d(16, 32, kernel_size=(4,), stride=(2,))
    (4): ReLU()
    (5): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
    (6): ReLU()
    (7): Flatten()
    (8): VariationalLayer(
      (mu): Linear(in_features=1152, out_features=15, bias=True)
      (rho): Linear(in_features=1152, out_features=15, bias=True)
      (softplus): Softplus(beta=1, threshold=20)
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=15, out_features=1152, bias=True)
    (1): ReLU()
    (2): Reshape()
    (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,))
    (4): ReLU()
    (5): ConvTranspose1d(32, 16, kernel_size=(4,), stride=(2,))
    (6): ReLU()
    (7): ConvTransposeDecoderOutput(
      (mu): ConvTranspose1d(16, 1, kernel_size=(6,), stride=(2,))
      (rho): Linear(in_features=1248, out_features=1, bias=True)
      (flatten): Flatten()
      (flatten_mu)

In [11]:
Z_hat, mu_tr, sigma_tr = CNNVAE2.encoder(X_train.to(device))
mu_train = mu_tr.cpu().detach().numpy()

Z_hat, mu_te, sigma_te = CNNVAE2.encoder(X_test.to(device))
mu_test = mu_te.cpu().detach().numpy()

In [12]:
CNNVAE2clf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, subsample=0.7, verbose=True)

In [13]:
CNNVAE2clf.fit(mu_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       39904.6121        2286.1505            6.21m
         2       37361.3994        1141.3668            6.11m
         3       35176.9283         852.4612            6.18m
         4       33788.4419         694.1255            6.14m
         5       32312.6260         554.5510            6.22m
         6       31312.5589         456.9539            6.30m
         7       30368.0183         418.4385            6.36m
         8       29482.5270         346.5589            6.35m
         9       28838.6219         283.3215            6.36m
        10       28201.4440         260.9410            6.38m
        20       24525.1604         111.6290            6.39m
        30       22534.1276          37.7495            6.12m
        40       21613.5756          30.3992            5.86m
        50       20968.5069          17.0853            5.69m
        60       20276.2680          23.2365            5.54m
       

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [14]:
y_pred = CNNVAE2clf.predict(mu_test)

In [15]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2850, 1238,   24,  123],
       [ 390, 9879,   39,  137],
       [  15,   48,  400,   16],
       [ 184,  175,   15, 1203]], dtype=int64)

In [17]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.856357552581262

In [18]:
dd.save("VAE-saved_model_and_losses/GB_on_CNNVAE-relu_latent_space.h5", CNNVAE2clf)

# LSTM VAE 

In [9]:
LSTMVAE = torch.load("VAE-saved_model_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300.pt")

In [10]:
LSTMVAE.eval()

DeepLSTM_VAE(
  (encoder_reshape): Reshape()
  (encoder_lstm): LSTM(1, 128)
  (encoder_output): VariationalLayer(
    (mu): Linear(in_features=128, out_features=15, bias=True)
    (rho): Linear(in_features=128, out_features=15, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
  (decoder_hidden): Linear(in_features=15, out_features=128, bias=True)
  (decoder_lstm): LSTM(1, 128)
  (decoder_output): RecurrentDecoderOutput(
    (mu): Linear(in_features=128, out_features=1, bias=True)
    (rho): Linear(in_features=20480, out_features=1, bias=True)
    (softplus): Softplus(beta=1, threshold=20)
  )
)

In [11]:
X_1 = X_train[:20000]
X_2 = X_train[20000:50000]
X_3 = X_train[50000:]

In [12]:
X_1.shape

torch.Size([20000, 160])

In [13]:
X_2.shape

torch.Size([30000, 160])

In [14]:
Z_hat, mu_tr, sigma_tr = LSTMVAE.encoder(X_1.to(device))
mu_train1 = mu_tr.cpu().detach().numpy()


In [15]:
#np.save("mu_lstm1.npy", mu_train1)

In [16]:
Z_hat, mu_tr, sigma_tr = LSTMVAE.encoder(X_2.to(device))
mu_train2 = mu_tr.cpu().detach().numpy()

In [17]:
#np.save("mu_lstm2.npy", mu_train2)

In [18]:
Z_hat, mu_tr, sigma_tr = LSTMVAE.encoder(X_3.to(device))
mu_train3 = mu_tr.cpu().detach().numpy()


In [19]:
#np.save("mu_lstm3.npy", mu_train3)

In [20]:
Z_hat, mu_te, sigma_te = LSTMVAE.encoder(X_test.to(device))
mu_test = mu_te.cpu().detach().numpy()

In [21]:
#np.save("mu_lstm_test.npy", mu_test)

In [25]:
mu_train = np.concatenate((mu_train1, mu_train2, mu_train3))

In [26]:
mu_train.shape

(66944, 15)

In [27]:
LSTMVAEclf = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, subsample=0.7, verbose=True)

In [28]:
LSTMVAEclf.fit(mu_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       39370.3342        2518.2006            5.89m
         2       36502.8299        1228.1537            5.97m
         3       34415.3546         888.0078            6.05m
         4       32781.7957         690.5433            6.09m
         5       31575.2621         556.3595            6.06m
         6       30395.7233         457.0149            6.06m
         7       29622.3986         377.4901            6.03m
         8       28769.6551         292.4901            6.01m
         9       28104.7339         310.2282            5.98m
        10       27638.7648         255.4609            5.96m
        20       23733.3450          92.9769            5.97m
        30       22233.2837          53.6143            5.80m
        40       21343.4186          26.9819            5.71m
        50       20630.5343          13.7826            5.59m
        60       20231.9677           9.2029            5.42m
       

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.7, tol=0.0001,
                           validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [29]:
y_pred = LSTMVAEclf.predict(mu_test)

In [30]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[2544, 1523,   33,  135],
       [ 480, 9780,   48,  137],
       [  23,   37,  402,   17],
       [ 161,  162,   22, 1232]], dtype=int64)

In [31]:
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8340105162523901

In [32]:
dd.save("VAE-saved_model_and_losses/GB_on_LSTM-VAE_latent_space.h5", LSTMVAEclf)